In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm

pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [2]:
from imblearn.over_sampling import SMOTE

## Train Test 1- All

In [9]:
df = pd.read_csv('../../data/processed/WaterUpdated.csv').drop('id', axis = 1)
df.target.replace({'functional': 0, 'non functional': 1, 'functional needs repair': 2}, inplace = True)
new_df = df
new_df = df.join(pd.get_dummies(df.basin, prefix = 'basin'))
new_df = new_df.join(pd.get_dummies(df.extraction_type, prefix = 'extract'))
new_df = new_df.join(pd.get_dummies(df.quantity, prefix = 'quantity'))
new_df = new_df.join(pd.get_dummies(df.water_quality, prefix = 'quality'))
new_df = new_df.join(pd.get_dummies(df.source, prefix = 'source'))
new_df = new_df.join(pd.get_dummies(df.waterpoint_type, prefix = 'waterpoint'))


unique_basin = [f'basin_{i}' for i in df.basin.unique()]
unique_extract = [f'extract_{i}' for i in df.extraction_type.unique()]
unique_waterpoint = [f'waterpoint_{i}' for i in df.waterpoint_type.unique()]
unique_source = [f'source_{i}' for i in df.source.unique() if i != 'unknown']
unique_quality = [f'quality_{i}' for i in df.water_quality.unique() if i != 'unknown']
unique_quantity = [f'quantity_{i}' for i in df.quantity.unique() if i != 'unknown']

col = ['amount_tsh', 'gps_height', 'population', 'permit', 'time_passed', 'target']
col = col + unique_basin + unique_extract + unique_waterpoint + unique_source + unique_quality + unique_quantity 
new_df = new_df[col]

print(new_df.target.value_counts())



X = new_df[col]
y = new_df[['target']]

x_train, x_test, y_train, y_test = train_test_split(X,y, stratify = X.target.values, train_size = .85, random_state = 0)


print(y_train.target.value_counts(), y_test.target.value_counts())

# func_df = x_train[x_train.target == 0]
# non_func = x_train[x_train.target ==1]
# needs_repair = x_train[x_train.target == 2]



x_train = x_train.drop('target', axis = 1)
x_test = x_test.drop('target', axis  = 1)

smote= SMOTE()
x_train_resamp, y_train_resamp = smote.fit_sample(x_train.values, y_train.target.values)

x_train_resamp = pd.DataFrame(x_train_resamp, columns = x_train.columns)
y_train_resamp = pd.DataFrame(y_train_resamp, columns = ['target'])
pickle.dump(y_test, open('../../data/processed/pickles/SMOTE_y_test.p', 'wb'))
pickle.dump(y_train_resamp, open('../../data/processed/pickles/SMOTE_y_train.p', 'wb'))
pickle.dump(x_train_resamp, open('../../data/processed/pickles/SMOTE_x_train.p', 'wb'))
pickle.dump(x_test, open('../../data/processed/pickles/SMOTE_x_test.p', 'wb'))


0    31899
1    21076
2     4272
Name: target, dtype: int64
0    27114
1    17914
2     3631
Name: target, dtype: int64 0    4785
1    3162
2     641
Name: target, dtype: int64


In [4]:
# X = resampled_concat
# y = resampled_concat[['target']]

# print(len(X), len(new_df))

# pickle.dump(X.drop('target', axis = 1), open('../../data/processed/pickles/X.p', 'wb'))
# pickle.dump(y, open('../../data/processed/pickles/y.p', 'wb'))

# x_train, x_test, y_train, y_test = train_test_split(X,y, stratify = X.target, random_state = 10, train_size = .85)
# print(y_train.target.value_counts(), y_test.target.value_counts())
# x_train = x_train.drop('target', axis =1)
# x_test = x_test.drop('target', axis =1)

# pickle.dump(y_test, open('../../data/processed/pickles/y_test.p', 'wb'))
# pickle.dump(y_train, open('../../data/processed/pickles/y_train.p', 'wb'))
# pickle.dump(x_train, open('../../data/processed/pickles/x_train.p', 'wb'))
# pickle.dump(x_test, open('../../data/processed/pickles/x_test.p', 'wb'))